In [11]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery
import re

In [240]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [241]:
#Nombre de lignes totales dans le JSON : 1320229

In [12]:
#intégration des credentials
from google.oauth2 import service_account

project = "marbotic"
key_path = "/Users/lauredegrave/code/casicoco/gcp/marbotic-310a9a33f756.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [ ]:
CHUNKSIZE = 100000
table_id = "marbotic_dataset.user_properties"

file_path = '../marbotic/data/goodformat.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["user_properties"].to_list()).copy()

    for col_concat in ['Products', 'Pieces', 'Games']:
        cat = ep[f'{col_concat}'].copy().map(lambda x: ", ".join(x)
                                            if isinstance(x, list) else x)
        names = list(
            set([
                x.strip().strip("'").strip("['")
                for x in ', '.join(', '.join(', '.join(
                    list(set([str(x) for x in cat]))).split('\n')).split(
                        "' '")).split(',')
            ]))
        names_transf = [
            col_concat + '_' + x.replace(' ', '_') for x in names
            if x != 'nan' and x != ''
        ]
        names = [x for x in names if x != 'nan' and x != '']
        for ind, name in enumerate(names_transf):

            ep[f'{name}'] = ep[f'{col_concat}'].map(
                lambda x: 1 if isinstance(x, str) and f'{names[ind]}' in x else
                (1 if isinstance(x, list) and
                 len([n for n in x if f'{names[ind]}' in n]) > 0
                 else 0))
            #ep['test']=ep[f'{col_concat}'].str.replace('[^\w\s\d]', '', regex=True)

    ep.drop(['Products', 'Pieces', 'Games'], axis=1, inplace=True)
    ep['event_id']=c["event_id"].copy()
    ep['id'] = (i*CHUNKSIZE) + index_init
    job = client.load_table_from_dataframe(ep, table_id)
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
        )


In [ ]:
columns_to_add_manually = ['Products_Sensory_kit']
query = f"DELETE FROM `{table_id}` WHERE true"
query_job = client.query(query)
results=query_job.result().to_dataframe()
results